In [32]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
import time
import datetime
from collections import Counter

df = pd.read_csv('../NYTimesBlogTrain.csv')
test_df = pd.read_csv('../NYTimesBlogTest.csv')

y = df['Popular'] 

# convert date to timestamp (milliseconds, divide by a million to get a more readable decimal)
df['PubDate'] = df['PubDate'].apply(lambda x: time.mktime(time.strptime(x, '%Y-%m-%d %H:%M:%S'))/1000000)
test_df['PubDate'] = test_df['PubDate'].apply(lambda x: time.mktime(time.strptime(x, '%Y-%m-%d %H:%M:%S'))/1000000)

#replace empty values with the middle most common word
def nan_to_meaningful(x, col):
    if pd.isnull(x):
        mc = Counter(col).most_common(8)
        mc = [str(i[0]) for i in mc]
        if 'nan' in mc:
            mc.remove('nan')
        rep = mc.pop((len(mc))//2)
        return "N/A"
        
    return x
        
my_list = ["NewsDesk","SectionName","SubsectionName","Headline","Snippet","Abstract"]

for f in my_list: 
    df[f] = df[f].apply(nan_to_meaningful, args=(df[f],))
    test_df[f] = test_df[f].apply(nan_to_meaningful, args=(test_df[f],))

# split sentences into series of words so each word can be a column
def string_to_series_of_words(x):    
    return pd.Series(x.split(" "))

# only apply it to the last 3 features which are sentences, index 3 to last
for f in my_list[3:]:
    df[f] = df[f].apply(string_to_series_of_words)
    test_df[f] = test_df[f].apply(string_to_series_of_words)


# for both testing/training data generate columns in the form of feature_word so that each word is a column
df=pd.get_dummies(df,columns=["NewsDesk","SectionName","SubsectionName", "Headline","Snippet","Abstract"],
                  drop_first=False)

test_df=pd.get_dummies(test_df,columns=["NewsDesk","SectionName","SubsectionName", "Headline","Snippet","Abstract"],
                  drop_first=False)

# we now have new features added, get them and remove the label
new_features = list(df.dtypes.index)
new_features.remove('Popular')

#do the same for the testing data
new_features_test = list(test_df.dtypes.index)

# Make sure both the training and testing dataframe have the same number of columns/features
# add the columns not in the other with all zeros

for f in new_features:
    if f not in new_features_test:
        test_df[f] = 0
        
new_features_test = list(test_df.dtypes.index)
        
for g in new_features_test:
    if g not in new_features:
        df[g] = 0
        
new_features = list(df.dtypes.index)
new_features.remove('Popular')
new_features.remove('UniqueID')
new_features_test.remove('UniqueID')
        
X = df[new_features]
Y = test_df[new_features_test]

#make sure at the end the testing and training dataframe have the same number of features
print(len(new_features))
print(len(new_features_test))



print("Done with data munging...")

6553
6553
Done with data munging...


In [38]:
from sklearn.ensemble import RandomForestClassifier

rf =  RandomForestClassifier(n_estimators = 50, bootstrap = True, random_state=0, criterion='entropy')
rf.fit(X, y)

y_predict_randF = rf.predict_proba(Y)

np.set_printoptions(threshold=np.inf)

print(np.array(y_predict_randF))



[[ 0.98  0.02]
 [ 0.88  0.12]
 [ 0.8   0.2 ]
 [ 0.84  0.16]
 [ 0.66  0.34]
 [ 0.66  0.34]
 [ 0.92  0.08]
 [ 0.92  0.08]
 [ 0.92  0.08]
 [ 0.98  0.02]
 [ 0.98  0.02]
 [ 0.84  0.16]
 [ 0.96  0.04]
 [ 0.66  0.34]
 [ 0.94  0.06]
 [ 0.64  0.36]
 [ 0.66  0.34]
 [ 0.92  0.08]
 [ 0.66  0.34]
 [ 0.78  0.22]
 [ 0.8   0.2 ]
 [ 0.94  0.06]
 [ 0.66  0.34]
 [ 0.94  0.06]
 [ 0.96  0.04]
 [ 0.7   0.3 ]
 [ 0.98  0.02]
 [ 0.98  0.02]
 [ 0.8   0.2 ]
 [ 0.96  0.04]
 [ 0.98  0.02]
 [ 0.92  0.08]
 [ 0.92  0.08]
 [ 0.88  0.12]
 [ 0.92  0.08]
 [ 0.88  0.12]
 [ 0.96  0.04]
 [ 0.96  0.04]
 [ 0.86  0.14]
 [ 0.96  0.04]
 [ 0.98  0.02]
 [ 0.58  0.42]
 [ 0.92  0.08]
 [ 0.72  0.28]
 [ 0.94  0.06]
 [ 0.98  0.02]
 [ 0.9   0.1 ]
 [ 0.96  0.04]
 [ 0.94  0.06]
 [ 0.9   0.1 ]
 [ 0.68  0.32]
 [ 0.86  0.14]
 [ 0.86  0.14]
 [ 0.78  0.22]
 [ 0.92  0.08]
 [ 0.94  0.06]
 [ 0.92  0.08]
 [ 0.9   0.1 ]
 [ 0.78  0.22]
 [ 0.98  0.02]
 [ 0.98  0.02]
 [ 0.92  0.08]
 [ 0.98  0.02]
 [ 0.96  0.04]
 [ 0.96  0.04]
 [ 0.98  0.02]
 [ 0.94  0

In [34]:
decTree =  DecisionTreeClassifier(criterion='entropy', min_samples_split=4, random_state=0)
decTree.fit(X, y)
y_predict_decTree = decTree.predict(Y)

print(np.array(y_predict_decTree))



[0 0 0 0 1 1 1 1 0 0 0 1 0 0 0 1 1 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 1 1 0 1 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 1 0 0 

In [36]:
from sklearn.ensemble import ExtraTreesClassifier

etc = ExtraTreesClassifier(n_estimators=52, max_depth=None, min_samples_split=5, random_state=0, criterion='entropy')
etc.fit(X,y)

predictions_etc = etc.predict(Y)

print(np.array(predictions_etc))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 